# Pre-requirments

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Wed Jan 24 10:26:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-tdee7t61
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-tdee7t61
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7429 sha256=7e6671b6b8bf4550a0703c056d2f0244cfe44c9d9046cd4a5ae2aaf799bf0fc2
  Stored in directory: /tmp/pip-ephem-wheel-cache-50akgurn/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [5]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpfu8r7wfz".


In [7]:
!apt-get install libglm-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglm-dev is already the newest version (0.9.9.8+ds-2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [8]:
!apt-get install libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [9]:
!apt-get install libgl1-mesa-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev
  libglx-dev libopengl-dev
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 220 kB of archives.
After this operation, 2,576 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libegl-dev amd64 1.4.0-1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgles1 amd64 1.4.0-1 [11.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 libg

# Ray Tracing Code

In [10]:
%%writefile main.cpp

Writing main.cpp


In [26]:
%%cuda_group_save --name "/content/graph.h" --g "g1"
#pragma once
# include <thrust/host_vector.h>
# include <thrust/device_vector.h>
# include <iostream>
# include <glm/glm.hpp>
# include <vector>

#define SCENE_MAX_SIZE 4
using vec3 = glm::vec3;

constexpr float infinity = std::numeric_limits<float>::infinity();
const vec3 camera_pos  = vec3(0., 0.35, -1.);
const vec3 light_point = vec3(5., 5., -10.);
const vec3 light_color = vec3(1., 1., 1.);
const float ambient    = 0.05;

__constant__ const vec3 O = vec3(0., 0.35, -1.);
__constant__ const vec3 light_point_cuda = vec3(5., 5., -10.);
__constant__ const vec3 light_color_cuda = vec3(1., 1., 1.);
__constant__ const float ambient_cuda = 0.05;

__host__ __device__ vec3 normalize(const vec3 &x);

class Object {
public:

    const vec3 position;
    const vec3 color;
    const float reflection;
    const float diffuse;
    const float specular_c;
    const float specular_k;

    __host__ __device__ Object(
        vec3 position,
        vec3 color,
        float reflection,
        float diffuse,
        float specular_c,
        float specular_k
    );

    __host__ __device__ virtual float intersect(const vec3& origin, const vec3& dir) = 0;
    __host__ __device__ virtual vec3 get_normal(const vec3& point) = 0;
    __host__ __device__ vec3 get_color();
    __host__ __device__ virtual ~Object() {}
};

class Sphere : public Object {
public:

    const float radius;

    __host__ __device__ Sphere(
        vec3 position,
        float radius,
        vec3 color,
        float reflection = .85,
        float diffuse = 1.,
        float specular_c = .6,
        float specular_k = 50
    );

    __host__ __device__ float intersect(const vec3& origin, const vec3& dir) override;

    __host__ __device__ vec3 get_normal(const vec3& point) override;
};

class Plane : public Object {
public:

    const vec3 normal;

    __host__ __device__ Plane(
        vec3 position,
        vec3 normal,
        vec3 color = vec3(1., 1., 1.),
        float reflection = 0.15,
        float diffuse = .75,
        float specular_c = .3,
        float specular_k = 50
    );

    __host__ __device__ float intersect(const vec3& origin, const vec3& dir) override;

    __host__ __device__ vec3 get_normal(const vec3& point) override;
};

void rendering_cuda(int w, int h, std::string filename);

__global__ void processImageKernel(int w, int h, float intensity, Object** scene_list, float* output, size_t numObjects);

__device__ vec3 intersect_color_kernel(vec3 origin, vec3 dir, float intensity, Object** scene_list, size_t numObjects);

__global__ void create_world(Object** d_list);

__global__ void free_scene(Object** d_list);

__device__ vec3 intersect_color_kernel_iter(vec3 origin, vec3 dir, float intensity, Object** scene_list, size_t numObjects);

In [27]:
%%cuda_group_save --name "/content/graph.cu" --g "g1"
# include "graph.h"
# include <thrust/host_vector.h>
# include <thrust/device_vector.h>
# include <cmath>
# include <limits>
# include <opencv2/opencv.hpp>

__host__ __device__ Object::Object(
    vec3 position,
    vec3 color,
    float reflection,
    float diffuse,
    float specular_c,
    float specular_k
): position(position), color(color), reflection(reflection), diffuse(diffuse), specular_c(specular_c), specular_k(specular_k) {}

__host__ __device__ vec3 Object::get_color() { return color; }

/* class Sphere */
__host__ __device__ Sphere::Sphere(
    vec3 position,
    float radius,
    vec3 color,
    float reflection,
    float diffuse,
    float specular_c,
    float specular_k
): Object(position, color, reflection, diffuse, specular_c, specular_k), radius(radius) {}

__host__ __device__ float Sphere::intersect(const vec3& origin, const vec3& dir) {

    const vec3 OC = position - origin;

    if (glm::length(OC) < radius || glm::dot(OC, dir) < 0) return infinity;

    float l        = glm::length(glm::dot(OC, dir));
    float m_square = glm::length(OC) * glm::length(OC) - l * l;
    float q_square = radius * radius - m_square;

    return (q_square >= 0) ? (l - sqrt(q_square)) : infinity;

}

__host__ __device__ vec3 Sphere::get_normal(const vec3& point) { return glm::normalize(point - position); }

/* class Plane */
__host__ __device__ Plane::Plane(
    vec3 position,
    vec3 normal,
    vec3 color,
    float reflection,
    float diffuse,
    float specular_c,
    float specular_k
): Object(position, color, reflection, diffuse, specular_c, specular_k), normal(normal) {}

__host__ __device__ float Plane::intersect(const vec3& origin, const vec3& dir) {

    float dn = glm::dot(dir, normal);

    if (abs(dn) < 1e-6) { return infinity; }

    float d = glm::dot(position - origin, normal) / dn;

    return d > 0 ? d : infinity;
}

__host__ __device__ vec3 Plane::get_normal(const vec3& point) { return normal; }

__host__ __device__ vec3 normalize(const vec3 &x) { return glm::normalize(x); }

__device__ float findMinElement(float* arr, size_t size) {
    if (size == 0) return INFINITY;

    float minVal = arr[0];
    for (size_t i = 1; i < size; ++i) {
        if (arr[i] < minVal) {
            minVal = arr[i];
        }
    }
    return minVal;
}

__device__ vec3 intersect_color_kernel(vec3 origin, vec3 dir, float intensity, Object** scene_list, size_t numObjects){
    float min_distance = infinity;

    size_t obj_index=-1;
    for (size_t i = 0; i < numObjects; ++i) {
        float current_distance = scene_list[i]->intersect(origin, dir);
        if (current_distance < min_distance) {
            min_distance = current_distance;
            obj_index = i;
        }
    }

    if (min_distance == infinity || intensity < 0.01) return vec3(0., 0., 0.);

    Object* obj   = scene_list[obj_index];
    const vec3 P  = origin + dir * min_distance;
    const vec3 PL = glm::normalize(light_point_cuda - P);
    const vec3 PO = glm::normalize(origin - P);
    const vec3 N  = obj->get_normal(P);

    const vec3 color = obj->get_color();
    vec3 c = ambient * color;

    float l[SCENE_MAX_SIZE];
    size_t l_size = 0;
    for (size_t i = 0; i < numObjects; ++i) {
        if (i != obj_index) {
            float intersect = scene_list[i]->intersect(P + N * .0001f, PL);
            if (intersect < glm::length(PL)) {
                l[l_size++] = intersect;
            }
        }
    }

    float minElement = findMinElement(l, l_size);
    if (!(l_size  > 0 && minElement < glm::length(PL))) {
        c += obj->diffuse * max(glm::dot(N, PL), 0.f) * color * light_color_cuda;
        c += obj->specular_c * powf(max(glm::dot(N, glm::normalize(PL + PO)), 0.f), obj->specular_k) * light_color_cuda;
    }
    vec3 reflect_ray = dir - 2 * glm::dot(dir, N) * N;
    c += obj->reflection * intersect_color_kernel(P + N * .0001f, reflect_ray, obj->reflection * intensity, scene_list, numObjects);
    return glm::clamp(c, 0.f, 1.f);
}

__device__ vec3 intersect_color_kernel_iter(vec3 origin, vec3 dir, float intensity, Object** scene_list, size_t numObjects) {
    vec3 accumulated_color = vec3(0.0, 0.0, 0.0);
    vec3 current_dir = dir;
    vec3 current_origin = origin;
    float current_intensity = intensity;

    while (current_intensity > 0.01) {
        float min_distance = infinity;
        size_t obj_index = -1;

        // 寻找最近交点
        for (size_t i = 0; i < numObjects; ++i) {
            float current_distance = scene_list[i]->intersect(current_origin, current_dir);
            if (current_distance < min_distance) {
                min_distance = current_distance;
                obj_index = i;
            }
        }

        // 如果没有交点，终止循环
        if (min_distance == infinity) {
            break;
        }

        // 计算交点处的各种参数
        Object* obj = scene_list[obj_index];
        vec3 P = current_origin + current_dir * min_distance;
        vec3 PL = glm::normalize(light_point_cuda - P);
        vec3 PO = glm::normalize(origin - P);
        vec3 N = obj->get_normal(P);
        vec3 color = obj->get_color();
        vec3 c = ambient * color;

        // 阴影判断
        float l[SCENE_MAX_SIZE];
        size_t l_size = 0;
        for (size_t i = 0; i < numObjects; ++i) {
            if (i != obj_index) {
                float intersect = scene_list[i]->intersect(P + N * 0.0001f, PL);
                if (intersect < glm::length(PL)) {
                    l[l_size++] = intersect;
                }
            }
        }

        float minElement = findMinElement(l, l_size);
        if (!(l_size > 0 && minElement < glm::length(PL))) {
            c += obj->diffuse * max(glm::dot(N, PL), 0.f) * color * light_color_cuda;
            c += obj->specular_c * powf(max(glm::dot(N, glm::normalize(PL + PO)), 0.f), obj->specular_k) * light_color_cuda;
        }

        // 累积颜色值
        accumulated_color += c * current_intensity;

        // 更新反射光线和强度
        current_dir = dir - 2 * glm::dot(dir, N) * N;
        current_origin = P + N * 0.0001f;
        current_intensity *= obj->reflection;
    }

    return glm::clamp(accumulated_color, 0.f, 1.f);
}


__global__ void processImageKernel(int w, int h, float intensity, Object** scene_list, float* output, size_t numObjects){

    float r = float(w) / h;
    glm::vec4 S = glm::vec4(-1., -1. / r + .25, 1., 1. / r + .25);
    vec3 Q = vec3(0., 0., 0.);
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    Q.x = S.x + x * (S.z - S.x) / (w - 1);
    Q.y = S.y + (h - 1 - y) * (S.w - S.y) / (h - 1); // 翻轉 Y 座標
    vec3 color = intersect_color_kernel_iter(O, glm::normalize(Q-O), intensity, scene_list, numObjects);

    int index = (y * w + x) * 3;
    output[index] = color.x;
    output[index + 1] = color.y;
    output[index + 2] = color.z;
}

__global__ void create_world(Object** d_list) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *(d_list)   = new Sphere(vec3(.75, .1, 1.), .6, vec3(.8, .3, 0.));
        *(d_list+1) = new Sphere(vec3(-.3, .01, .2), .3, vec3(.0, .0, .9));
        *(d_list+2) = new Sphere(vec3(-2.75, .1, 3.5), .6, vec3(.1, .572, .184));
        *(d_list+3) = new Plane(vec3(0., -.5, 0.), vec3(0., 1., 0.));
    }
}

__global__ void free_scene(Object** d_list) {
    delete *(d_list);
    delete *(d_list+1);
    delete *(d_list+2);
    delete *(d_list+3);
}

void rendering_cuda(int w, int h, std::string filename) {

    cv::Mat img(h, w, CV_32FC3);

    /* CUDA */
    float* hostImage;
    size_t size = w * h * 3 * sizeof(float);
    cudaHostAlloc((void**)&hostImage, size, cudaHostAllocMapped);
    memcpy(hostImage, img.data, size);

    size_t numObjects = 4;
    Object** scene_list;
    size_t size2 = 4 * sizeof(Object*);
    cudaHostAlloc((void**)&scene_list, size2, cudaHostAllocMapped);
    create_world<<<1, 1>>>(scene_list);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((w + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (h + threadsPerBlock.y - 1) / threadsPerBlock.y);
    processImageKernel<<<numBlocks, threadsPerBlock>>>(w, h, 1,
                                                       scene_list,
                                                       hostImage, numObjects);
    cudaDeviceSynchronize();

    memcpy(img.data, hostImage, size);
    cudaFreeHost(hostImage);
    free_scene<<<1,1>>>(scene_list);
    img *= 255;
    img.convertTo(img, CV_8UC3);
    cv::imwrite(filename, img);
}

In [28]:
%%cuda_group_save --name "/content/main.cu" --g "g1"
# include <iostream>
# include <string>
# include <thrust/host_vector.h>
# include <thrust/device_vector.h>
# include "graph.h"
# include <glm/glm.hpp>
# include <cstdlib> // For exit()
# include <stdexcept> // For std::invalid_argument

int main(int argc, char *argv[]) {

    /* Process Usr Input */

    // int w = 1080, h = 960;
    int w = 7680, h = 4320;
    bool wSet = false, hSet = false;
    try{
        for (int i = 1; i<argc; i++ ) {
            std::string arg = argv[i];
            if (arg == "-w" && i + 1 < argc) {
                w = std::stoi(argv[++i]);
                wSet = true;
            }
            else if (arg == "-h" && i + 1 < argc) {
                h = std::stoi(argv[++i]);
                hSet = true;
            }
        }
    } catch (const std::invalid_argument& e) {
        std::cerr << "Error: Invalid argument for width or height." << std::endl;
        std::exit(EXIT_FAILURE);
    }

    if (wSet != hSet) {
        std::cerr << "Error: Both -w and -h must be provided together." << std::endl;
        std::exit(EXIT_FAILURE);
    }

    rendering_cuda(
        w, h,
        "result_cuda.png"
    );

    return 0;
}

In [20]:
!dpkg -L libglm-dev | grep /glm/glm.hpp
!dpkg -L libopencv-dev | grep /opencv2/opencv.hpp

/usr/include/glm/glm.hpp


In [29]:
!nvcc -o /content/main /content/main.cu /content/graph.cu -I/usr/include/glm -I/usr/include/opencv4 -lGL -lopencv_core -lopencv_imgproc -lopencv_highgui -lopencv_imgcodecs

/usr/include/glm/detail/type_vec2.hpp(94): warning #20012-D: __device__ annotation is ignored on a function("vec") that is explicitly defaulted on its first declaration
   __attribute__((device)) __attribute__((host)) 
                  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/glm/detail/type_vec2.hpp(94): warning #20012-D: __host__ annotation is ignored on a function("vec") that is explicitly defaulted on its first declaration
   __attribute__((device)) __attribute__((host)) 
                                          ^

/usr/include/glm/detail/type_vec2.hpp(95): warning #20012-D: __device__ annotation is ignored on a function("vec") that is explicitly defaulted on its first declaration
   __attribute__((device)) __attribute__((host)) 
                  ^

/usr/include/glm/detail/type_vec2.hpp(95): warning #20012-D: __host__ annotation is ignored on a function("vec") that is explicitly defaulted on its first declaration
   __attribu

In [31]:
!/content/main

# Compile & analyze

In [ ]:
!nvprof /content/src/main

In [ ]:
!nvprof --print-gpu-trace /content/src/main

In [ ]:
!rm /content/main
!rm result.png